<a href="https://colab.research.google.com/github/bruninho-design/AjudaOSul.AI-ImersaoAlura/blob/main/AjudaOSulAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Instalando o SDK do Google**

In [1]:
!pip install -q -U google-generativeai

# Importando as Lib

In [3]:
import google.generativeai as genai
from google.colab import userdata
api_key = userdata.get("SECRET_KEY")
genai.configure(api_key=api_key)

# Função para analisar a imagem e verificar autenticidade

In [7]:
def analisar_imagem(imagem_url):
    # Cria um cliente para a API Vision
    client = vision.ImageAnnotatorClient()

    # Faz a chamada à API para analisar a imagem
    response = client.annotate_image({
        'image': {'source': {'image_uri': imagem_url}},
        'features': [{'type': vision.Feature.Type.SAFE_SEARCH_DETECTION}],
    })

    # Verifica se a imagem é considerada segura
    safe_search = response.safe_search_annotation

    # Atribui pontos com base na análise da imagem
    if safe_search.adult == vision.Likelihood.VERY_UNLIKELY and safe_search.spoof == vision.Likelihood.VERY_UNLIKELY:
        return 30  # Imagem autêntica
    else:
        return 0  # Imagem suspeita

# Função para analisar a descrição do texto

In [8]:
def analisar_texto(descricao):
    # Cria um cliente para a API de Linguagem Natural
    client = language_v1.LanguageServiceClient()

    # Analisa o texto
    document = language_v1.Document(content=descricao, type_=language_v1.Document.Type.PLAIN_TEXT)
    response = client.analyze_sentiment(request={'document': document})

    # Extrai a pontuação de sentimento
    sentiment_score = response.document_sentiment.score

    # Analisa entidades no texto (ex: locais, pessoas)
    entities = client.analyze_entities(request={'document': document}).entities

    # Atribui pontos com base na análise do texto
    pontos = 0
    if sentiment_score > 0:  # Sentimento positivo indica maior necessidade
        pontos += 10
    if len(entities) > 0:  # Descrição mais detalhada com entidades
        pontos += 20
    # Adiciona pontos para palavras-chave relevantes (alagamento, desabrigado, etc.)
    for entity in entities:
        if entity.name.lower() in ["alagamento", "desabrigado", "socorro", "ajuda"]:
            pontos += 5

    return pontos

# Função para calcular o score do pedido de ajuda

In [6]:
def calculate_help_request_score(imagem_url, descricao, criancas):
    score_imagem = analisar_imagem(imagem_url)
    score_texto = analisar_texto(descricao)
    score_criancas = 20 if criancas else 0
    score_total = score_imagem + score_texto + score_criancas
    return score_total

# Exemplo de uso

In [ ]:

imagem_url = "https://exemplo.com/imagem.jpg"
descricao = "Perdi tudo no alagamento, estou desabrigado com minha família e precisamos de ajuda."
criancas = True

score = calculate_help_request_score(imagem_url, descricao, criancas)

print(f"Score do pedido de ajuda: {score}")

if score >= 70:
    print("Pedido de ajuda considerado legítimo.")
else:
    print("Pedido de ajuda requer investigação adicional.")

# Inicializando o chat

In [ ]:
chat = model.start_chat(history=[])

while True:
    # Solicitando informações do pedido de ajuda
    imagem = input("Digite o caminho da imagem do anúncio de ajuda (ou 'fim' para sair): ")
    if imagem == "fim":
        break
    descricao = input("Digite a descrição do anúncio de ajuda: ")
    criancas = input("Há crianças na família? (sim/não): ").lower() == "sim"

    # Calculando o score do pedido
    score = calculate_help_request_score(imagem, descricao, criancas)

    # Classificando o pedido e exibindo resultados
    if score >= 70:
        classificacao = "Legítimo"
    else:
        classificacao = "Suspeito (requer investigação)"

    print(f"\nScore: {score} - Classificação: {classificacao}")
    print("-" * 30)

    # Exibindo análise da IA (se disponível)
    prompt = [genai.upload_file(imagem),
              "Analise esta imagem e indique se as condições mostradas correspondem a um cenário real de crise climática no Rio Grande do Sul."]
    response = chat.send_message(prompt)
    print(f"**Análise da IA:** {response.parts[0].text}")
    print("-" * 30)

# Exibindo histórico do chat (opcional)